## 라이브러리 호출

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import torch

major_version, minor_version = torch.cuda.get_device_capability()
major_version, minor_version


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-package

(8, 0)

In [3]:
!pip install torch==2.1.0+cu118 torchvision==0.16.0+cu118 torchaudio==2.1.0+cu118 \
  -f https://download.pytorch.org/whl/torch_stable.html

# 3. flash-attn 설치
!pip install flash-attn --no-build-isolation --no-deps --index-url https://pypi.nvidia.com

!pip install packaging ninja einops xformers trl peft accelerate bitsandbytes


Looking in links: https://download.pytorch.org/whl/torch_stable.html
  Using cached triton-2.1.0-0-cp311-cp311-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.3 kB)
Using cached triton-2.1.0-0-cp311-cp311-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (89.2 MB)
  Attempting uninstall: triton
    Found existing installation: triton 3.3.0
    Uninstalling triton-3.3.0:
      Successfully uninstalled triton-3.3.0
Looking in indexes: https://pypi.nvidia.com
ERROR: Could not find a version that satisfies the requirement flash-attn (from versions: none)
ERROR: No matching distribution found for flash-attn
  Using cached xformers-0.0.30-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (1.0 kB)
  Using cached trl-0.19.0-py3-none-any.whl.metadata (10 kB)
  Using cached bitsandbytes-0.46.0-py3-none-manylinux_2_24_x86_64.whl.metadata (10 kB)
  Using cached torch-2.7.0-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (29 kB)
  Using cached nvidia_cudnn_cu12-9.5.1.17-py3-none-manylinux_2_28_

In [4]:
!pip install unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 1.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.9/277.9 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.1/149.1 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 93.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.3/128.3 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.7/210.7 kB 20.6 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.16.0+cu118
    Uninstalling torchvision-0.16.

In [1]:

import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import TrainingArguments, BitsAndBytesConfig
from trl import SFTTrainer
import torch
from unsloth import FastLanguageModel
from datasets import Dataset
from tqdm.auto import tqdm
import wandb
import numpy as np
import random
import os
import requests

CFG = {
    'MODEL_NAME': "mistralai/Magistral-Small-2506",
    'LEARNING_RATE': 1e-4,
    'STEPS': 2000,
    'SEED': 42,
    'exp_name': 'exp19'
}

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)

set_seed(CFG['SEED'])

train = pd.read_csv('/content/drive/MyDrive/CONTEXT/data/train.csv')

def make_input(row):
    sentences = [row[f"sentence_{i}"] for i in range(4)]
    input_text = "문장을 순서대로 정렬하세요: " + " </s> ".join(sentences)
    answer = [row[f"answer_{i}"] for i in range(4)]
    target_text = " ".join(map(str, answer))
    return {"input": input_text, "target": target_text}

inputs = train.apply(make_input, axis=1).tolist()

def compute_accuracy(preds, labels):
    correct = 0
    total = 0
    for p, l in zip(preds, labels):
        if p == l:
            correct += 1
        total += 1
    return correct / total if total > 0 else 0

def predict_order(sent_list):
    prompt = sent_list + "\n정답:"
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        padding="longest",
        max_length=512
    ).to(model.device)
    prompt_len = inputs["input_ids"].shape[1]
    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=8,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    answer = decoded.split("정답:")[-1].strip().replace('\n', ' ')
    try:
        order = list(map(int, answer.strip().split()))
        return order
    except ValueError as e:
        return [0, 1, 2, 3]

train_data = inputs
train_dataset = Dataset.from_pandas(pd.DataFrame(train_data))

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name= CFG['MODEL_NAME'],
    max_seq_length=4096,
    dtype=None,
    load_in_4bit = True,
)

model = FastLanguageModel.get_peft_model(
    model,
    r=32,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=42,
    use_rslora=False,
    loftq_config=None,
)

def tokenize(batch):
    prompts = [inp.strip() + "\n정답:" for inp in batch["input"]]
    full_texts = [p + " " + tgt + tokenizer.eos_token
                for p, tgt in zip(prompts, batch["target"])]
    tok = tokenizer(full_texts,
                    max_length=512,
                    truncation=True,
                    padding="max_length")
    labels = []
    for prompt, ids in zip(prompts, tok["input_ids"]):
        prompt_len = len(tokenizer(prompt, add_special_tokens=False)["input_ids"])
        lbl = ids.copy()
        lbl[:prompt_len] = [-100] * prompt_len
        labels.append(lbl)
    tok["labels"] = labels
    return tok

tokenized_train = train_dataset.map(tokenize, batched=True)

from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=tokenized_train,
    dataset_num_proc=8,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        num_train_epochs=3,
        logging_steps=1,
        learning_rate=CFG['LEARNING_RATE'],
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="cosine",
        seed=CFG['SEED'],
        output_dir=f"/content/drive/MyDrive/CONTEXT/model/{CFG['exp_name']}",
    ),
)
trainer.train()

model.eval()

test = pd.read_csv("/content/drive/MyDrive/CONTEXT/data/test.csv")

def make_input(row):
    sentences = [row[f"sentence_{i}"] for i in range(4)]
    input_text = "문장을 순서대로 정렬하세요: " + " </s> ".join(sentences)
    return input_text

test_inputs = test.apply(make_input, axis=1).tolist()

def predict_order(sent_list):
    prompt = sent_list + "\n정답:"
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        padding="longest",
        max_length=512
    ).to(model.device)
    prompt_len = inputs["input_ids"].shape[1]
    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=8,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    answer = decoded.split("정답:")[-1].strip().replace('\n', ' ')
    try:
        order = list(map(int, answer.strip().split()))
        return order
    except ValueError as e:
        return [0, 1, 2, 3]

predictions = []
for sent_group in tqdm(test_inputs, desc="Predicting"):
    pred = predict_order(sent_group)
    predictions.append(pred)

sample_submission = pd.read_csv("/content/drive/MyDrive/CONTEXT/data/sample_submission.csv")

for i in range(4):
    sample_submission[f"answer_{i}"] = [
        pred[i] if len(pred) == 4 else i for pred in predictions
    ]

sample_submission.to_csv(f"/content/drive/MyDrive/CONTEXT/submission/{CFG['exp_name']}.csv", index=False)


/tmp/ipython-input-1-3646125585.py:17: UserWarning: WARNING: Unsloth should be imported before trl, transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel  # 고속 언어 모델 구현 라이브러리


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sungcoook (sungcook) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


✅ 메시지 전송 완료: {'ok': True, 'result': {'message_id': 59, 'from': {'id': 7913886213, 'is_bot': True, 'first_name': 'goorm _bot', 'username': 'sungkookbot'}, 'chat': {'id': 6660461954, 'first_name': 'A', 'last_name': 'Sj', 'type': 'private'}, 'date': 1750467800, 'text': '[exp19] mistralai/Magistral-Small-2506\n✅ 데이터 로딩 완료 - 학습 데이터: 7351개'}}
==((====))==  Unsloth 2025.6.3: Fast Mistral patching. Transformers: 4.52.4.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/164k [00:00<?, ?B/s]

...

wandb: WARNING The get_url method is deprecated and will be removed in a future release. Please use `run.url` instead.


✅ 메시지 전송 완료: {'ok': True, 'result': {'message_id': 63, 'from': {'id': 7913886213, 'is_bot': True, 'first_name': 'goorm _bot', 'username': 'sungkookbot'}, 'chat': {'id': 6660461954, 'first_name': 'A', 'last_name': 'Sj', 'type': 'private'}, 'date': 1750480581, 'text': '[exp19] mistralai/Magistral-Small-2506\n✅ 학습 완료'}}
✅ 메시지 전송 완료: {'ok': True, 'result': {'message_id': 64, 'from': {'id': 7913886213, 'is_bot': True, 'first_name': 'goorm _bot', 'username': 'sungkookbot'}, 'chat': {'id': 6660461954, 'first_name': 'A', 'last_name': 'Sj', 'type': 'private'}, 'date': 1750480582, 'text': '[exp19] mistralai/Magistral-Small-2506\n✅ 추론 시작'}}


Predicting:   0%|          | 0/1780 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.

✅ 메시지 전송 완료: {'ok': True, 'result': {'message_id': 65, 'from': {'id': 7913886213, 'is_bot': True, 'first_name': 'goorm _bot', 'username': 'sungkookbot'}, 'chat': {'id': 6660461954, 'first_name': 'A', 'last_name': 'Sj', 'type': 'private'}, 'date': 1750482007, 'text': '[exp19] mistralai/Magistral-Small-2506\n✅ 추론 완료'}}
